### Test Web_Scrapping

#### Installation des packages necessaires

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
import json
import urllib
import sys
import time

# adding path to geckodriver to the OS environment variable
os.environ["PATH"] += os.pathsep + os.getcwd()

#### Configuration des variables

In [4]:
# Configuration
download_path = "database/" # Chemin relatif dans lequel les images vont être enregistrées

# Images
words_to_search = ['hieroglyphe'] # Mot clé / Mots clés à chercher pour telecharger les images
nb_to_download = [10] # Nombre d'images à enregistrer
first_image_position = [0] # Numéro de l'image à partir de laquelle il faut télécharger

In [5]:
def main():
    if len(words_to_search) != len(nb_to_download) or len(nb_to_download) != len(first_image_position) :
        raise ValueError('You may have forgotten to configure one of the lists (length is different)')
    i= 0
    # For each word in the list, we download the number of images requested
    while i<len(words_to_search):
        print("Words "+str(i)+" : "+str(nb_to_download[i])+"x\""+words_to_search[i]+"\"")
        if nb_to_download[i] > 0:
            search_and_save(words_to_search[i],nb_to_download[i], first_image_position[i])
        i+=1

#### Définition de la fonction qui cherche puis enregistre les images dans un fichier

In [6]:
def search_and_save(text, number, first_position):
    # Number_of_scrolls * 400 images will be opened in the browser
    number_of_scrolls = int((number + first_position)/ 400 + 1) 
    print("Search : "+text+" ; number : "+str(number)+"; first_position : "+str(first_position)+" ; scrolls : "+str(number_of_scrolls))

    # Create directories to save images
    if not os.path.exists(download_path + text.replace(" ", "_")):
        os.makedirs(download_path + text.replace(" ", "_"))

    # Connect to Google Image
    #url = "https://www.google.co.in/search?q="+text+"&source=lnms&tbm=isch"
    url="https://www.google.fr/search?q=HI%C3%89ROGLYPHE&tbm=isch&hl=fr&hl=fr&tbs=ic%3Aspecific%2Cisc%3Aorange&ved=0CAUQ2J8EahcKEwiomIuP9rfnAhUAAAAAHQAAAAAQAg&biw=1440&bih=740"
    driver = webdriver.Chrome('database/chromedriver')
    driver.get(url)
    headers = {}
    headers['User-Agent'] = "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"
    extensions = {"jpg", "jpeg", "png", "gif"}
    
    img_count = 0
    downloaded_img_count = 0
    img_skip = 0
    
    #Prepare Google Page
    for _ in range(number_of_scrolls):
        for __ in range(10):
            # Multiple scrolls needed to show all 400 images
            driver.execute_script("window.scrollBy(0, 1000000)")
            time.sleep(0.2)
        # to load next 400 images
        time.sleep(2.5)
        try:
            driver.find_element_by_xpath("//input[@value='Show more results']").click()
            time.sleep(2.5)
        except Exception as e:
            print("Less images found:"+ str(e))
            break

    # Téléchargement des images
    imges = driver.find_elements_by_xpath('//div[contains(@class,"rg_meta")]')
    print("Total images:"+ str(len(imges)) + "\n")
    for img in imges:
        if img_skip < first_position:
            # Skip first images if asked to
            img_skip += 1
        else :
            # Get image
            img_count += 1
            img_url = json.loads(img.get_attribute('innerHTML'))["ou"]
            img_type = json.loads(img.get_attribute('innerHTML'))["ity"]
            print("Downloading image "+ str(img_count) + ": "+ img_url)
            try:
                if img_type not in extensions:
                    img_type = "jpg"
                # Download image and save it
                req = urllib.request.Request(img_url, headers=headers)
                raw_img = urllib.request.urlopen(req).read()
                f = open(download_path+text.replace(" ", "_")+"/"+str(img_skip+downloaded_img_count)+"."+img_type, "wb")
                f.write(raw_img)
                f.close
                downloaded_img_count += 1
            except Exception as e:
                print("Download failed:"+ str(e))
            finally:
                print("")
            if downloaded_img_count >= number:
                break

    print("Total skipped : "+str(img_skip)+"; Total downloaded : "+ str(downloaded_img_count)+ "/"+ str(img_count))
    driver.quit()

#if __name__ == "__main__":
#    main()

#### Mise en route de la fonction pour reccupérer les images

In [7]:
search_and_save("hieroglyphe", 30, 0)

Search : hieroglyphe ; number : 30; first_position : 0 ; scrolls : 1
Less images found:Message: no such element: Unable to locate element: {"method":"xpath","selector":"//input[@value='Show more results']"}
  (Session info: chrome=79.0.3945.130)

Total images:300































Total skipped : 0; Total downloaded : 30/30


#### Test du webdriver

In [11]:
driver = webdriver.Chrome('database/chromedriver')  # Optional argument, if not specified will search path.
driver.get('http://www.google.com/');
time.sleep(5) # Let the user actually see something!
search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.submit()
time.sleep(5) # Let the user actually see something!
driver.quit()